[View in Colaboratory](https://colab.research.google.com/github/sqlwu/image/blob/master/vgg16_vs.ipynb)

In [1]:
#https://ithelp.ithome.com.tw/articles/10192389
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np
import os
import sys

Using TensorFlow backend.


In [0]:
# 計算相似矩陣
def cosine_similarity(ratings):
    sim = ratings.dot(ratings.T)
    if not isinstance(sim, np.ndarray):
        sim = sim.toarray()
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)
  
 '''
 cosine_similarity 函數：這是程式最重要的部分，將images目錄內的每一張照片轉成特徵向量，再兩兩作比較，利用 cosine 函數計算兩個特徵向量的角度，越接近 1，表示越相似
 '''

In [55]:
ls #查看目錄

00.PNG  04.PNG  08.PNG  anscombe.json*                mnist_train_small.csv
01.PNG  05.PNG  09.PNG  california_housing_test.csv   README.md*
02.PNG  06.PNG  10.PNG  california_housing_train.csv
03.PNG  07.PNG  11.PNG  mnist_test.csv


In [0]:
cp /content/11.PNG /content/sample_data/ #複製檔案/content/11.PNG到/content/sample_data/

In [54]:
cd sample_data #前往目錄

/content/sample_data


In [56]:
cd .. #返回目錄

/content


In [50]:
# acquire data
from google.colab import files

uploads = files.upload()

Saving 10.PNG to 10.PNG
Saving 11.PNG to 11.PNG


In [0]:
def main():
    # 自 images 目錄找出所有 JPEG 檔案    
    y_test=[]
    x_test=[]
    for img_path in os.listdir("sample_data"):
        if img_path.endswith(".PNG"):
            img = image.load_img("sample_data/"+img_path, target_size=(224, 224))
            y_test.append(img_path[0:11])
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            if len(x_test) > 0:
                x_test = np.concatenate((x_test,x))
            else:
                x_test=x

    # 轉成 VGG 的 input 格式
    x_test = preprocess_input(x_test)

    # include_top=False，表示會載入 VGG16 的模型，不包括加在最後3層的卷積層，通常是取得 Features (1,7,7,512)
    model = VGG16(weights='imagenet', include_top=False)
    
    # 萃取特徵
    features = model.predict(x_test)
    # 計算相似矩陣
    features_compress = features.reshape(len(y_test),7*7*512)
    sim = cosine_similarity(features_compress)
    print(sim)

    # 隨機取1個樣本測試
    inputNo = np.random.randint(0,len(y_test),1)[0]
    
    # 依命令行參數，取1個樣本測試測試
    #inputNo = 11
    
    print(inputNo)
    top = np.argsort(-sim[inputNo], axis=0)[1:3]

    # 取得最相似的前2名序號
    recommend = [y_test[i] for i in top]
    print(recommend)

In [73]:
if __name__ == "__main__":
    main()

[[0.99999994 0.5629465  0.44928277 0.34270215 0.55493945 0.53685606
  0.3810024  0.6218299  0.15963304 0.16289735 0.4957732  0.4831454 ]
 [0.5629465  1.         0.41552377 0.35993838 0.5114108  0.5681614
  0.36036202 0.45214924 0.1515414  0.14417356 0.40312198 0.52209604]
 [0.44928277 0.41552377 1.0000001  0.3130089  0.3759732  0.40787247
  0.40678146 0.38796926 0.12866144 0.1722233  0.34614483 0.41588056]
 [0.34270215 0.35993838 0.3130089  1.         0.3511254  0.32127628
  0.3316117  0.33602905 0.14806597 0.15176034 0.29776466 0.3125978 ]
 [0.55493945 0.5114108  0.37597322 0.35112542 1.0000001  0.5404161
  0.3679068  0.51144785 0.14887248 0.16327682 0.52428025 0.49133354]
 [0.5368561  0.56816137 0.40787247 0.32127625 0.5404161  1.
  0.39207095 0.4949096  0.16590714 0.17243484 0.42163485 0.7044052 ]
 [0.3810024  0.36036202 0.40678146 0.3316117  0.3679068  0.39207095
  1.         0.35950652 0.12258141 0.1534804  0.30775294 0.37441325]
 [0.6218299  0.45214927 0.38796926 0.33602905 0.511

In [0]:
'''
main 函數：

先自 images 目錄找出所有 JPEG 檔案，讀出圖檔內容，將他們轉成 VGG input 格式。
使用 VGG 16 模型，include_top=False，表示會載入 VGG16 的模型，不包括加在最後3層的卷積層，只計算特徵(Features)向量。
呼叫 VGG 16 模型，將images目錄內的每一張照片萃取出特徵向量，再轉為一維，呼叫 cosine_similarity 函數，計算每一張照片與其他照片的相似度。
依命令行參數，取一張照片當作 input 樣本測試。
根據該樣本所在列查詢與其他圖檔的相似度，找出最相似的前2名序號。
'''